# 1-Imports


In [43]:
import numpy as np
from joblib import dump, load
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# 2-Load Features


In [44]:
X_train = np.load("../features/train_features.npy")
y_train = np.load("../features/train_labels.npy")

X_val = np.load("../features/val_features.npy")
y_val = np.load("../features/val_labels.npy")
print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)


Train: (3000, 98) (3000,)
Val:   (373, 98) (373,)


# 3-Sanity check

In [45]:
print("Feature min:", X_train.min())
print("Feature max:", X_train.max())
print("Mean:", X_train.mean())
print("Std:", X_train.std())


Feature min: 0.0
Feature max: 16.0
Mean: 0.9806422512050271
Std: 1.8744800547108122


# 4-SVM Pipeline

In [46]:
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='rbf', C=10.0, gamma='scale'))
])

# 5-Training SVM


In [47]:
print("Training SVM...")
svm_pipeline.fit(X_train, y_train)
print("Training done.")

Training SVM...
Training done.


# 6-Validation Evaluation

In [48]:
y_pred = svm_pipeline.predict(X_val)

acc = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", acc)

print("\nClassification Report:")
print(classification_report(y_val, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred))


Validation Accuracy: 0.7640750670241286

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.75      0.74        72
           1       0.80      0.81      0.81        91
           2       0.73      0.84      0.78        45
           3       0.85      0.79      0.82        81
           4       0.61      0.63      0.62        60
           5       0.89      0.71      0.79        24

    accuracy                           0.76       373
   macro avg       0.77      0.76      0.76       373
weighted avg       0.77      0.76      0.76       373


Confusion Matrix:
[[54  2  1  4  9  2]
 [ 5 74  4  2  6  0]
 [ 2  3 38  0  2  0]
 [ 3  6  3 64  5  0]
 [ 9  5  4  4 38  0]
 [ 0  2  2  1  2 17]]


# 7-Saving Model


In [49]:

dump(svm_pipeline, 'svm_pipeline.joblib')

loaded_pipeline = load('svm_pipeline.joblib')
